In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

# -- Already installed above --
import pandas as pd
import numpy as np
import re
import nltk
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Dropout, GlobalAveragePooling1D, Layer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# Download required nltk packages
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('omw-1.4')

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Projects/ProjectMonkeyPox/Monkeypox Dataset.csv')

# Text cleaning
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#", "", text)
    text = re.sub(r"[^A-Za-z\s]", "", text)
    return text.lower()

df["Cleaned_Text"] = df["Translated Post Description"].apply(clean_text)

lemmatizer = nltk.WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words("english"))

def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

df["Processed_Text"] = df["Cleaned_Text"].apply(preprocess_text)

# Sentiment remapping
sentiment_mapping = {
    "anger": "Negative",
    "sadness": "Negative",
    "neutral": "Neutral",
    "joy": "Positive"
}
df["Merged_Sentiment"] = df["Sentiment"].map(sentiment_mapping)

label_encoder = LabelEncoder()
df["Sentiment_Encoded"] = label_encoder.fit_transform(df["Merged_Sentiment"])

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df["Processed_Text"], df["Sentiment_Encoded"], test_size=0.2, stratify=df["Sentiment_Encoded"], random_state=42
)

--2025-04-14 09:47:58--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-04-14 09:47:58--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-04-14 09:47:58--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# Tokenization
max_words = 15000
max_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post", truncating="post")

# Load GloVe embeddings
embedding_dim = 100
embedding_index = {}
with open("glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector

# Embedding matrix
word_index = tokenizer.word_index
num_words = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i < num_words and word in embedding_index:
        embedding_matrix[i] = embedding_index[word]

# Custom Attention Layer
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")
        super().build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

# Model architecture
input_ = Input(shape=(max_len,))
x = Embedding(input_dim=num_words, output_dim=embedding_dim,
              weights=[embedding_matrix], input_length=max_len,
              trainable=False)(input_)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Attention()(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(len(label_encoder.classes_), activation="softmax")(x)

model = Model(inputs=input_, outputs=output)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Class weights
class_weights = compute_class_weight(class_weight="balanced",
                                     classes=np.unique(y_train),
                                     y=y_train)
class_weights = dict(enumerate(class_weights))

# Train model
early_stopping = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
model.fit(X_train_pad, y_train,
          validation_data=(X_test_pad, y_test),
          epochs=25, batch_size=32,
          class_weight=class_weights,
          callbacks=[early_stopping])

# Evaluate
y_pred = model.predict(X_test_pad)
y_pred = np.argmax(y_pred, axis=1)
print("Attention BiLSTM Classification Report")
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/25
411/411 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.5685 - loss: 0.9047 - val_accuracy: 0.7097 - val_loss: 0.6657
Epoch 2/25
411/411 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - accuracy: 0.7374 - loss: 0.6461 - val_accuracy: 0.7633 - val_loss: 0.5837
Epoch 3/25
411/411 ━━━━━━━━━━━━━━━━━━━━ 19s 26ms/step - accuracy: 0.7650 - loss: 0.5824 - val_accuracy: 0.7648 - val_loss: 0.5843
Epoch 4/25
411/411 ━━━━━━━━━━━━━━━━━━━━ 22s 29ms/step - accuracy: 0.7868 - loss: 0.5299 - val_accuracy: 0.7822 - val_loss: 0.5568
Epoch 5/25
411/411 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - accuracy: 0.8031 - loss: 0.5022 - val_accuracy: 0.7883 - val_loss: 0.5383
Epoch 6/25
411/411 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.8219 - loss: 0.4682 - val_accuracy: 0.8069 - val_loss: 0.5125
Epoch 7/25
411/411 ━━━━━━━━━━━━━━━━━━━━ 19s 26ms/step - accuracy: 0.8301 - loss: 0.4354 - val_accuracy: 0.8014 - val_loss: 0.5244
Epoch 8/25
411/411 ━━━━━━━━━━━━━━━━━━━━ 22s 29ms/step - accuracy: 0.8560 - loss: 0.3799 - 

In [ ]:
# Define path
model_path = "/content/drive/MyDrive/Projects/ProjectMonkeyPox/Saved_Bi-lstm/bilstm_attention_model.h5"

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(model_path), exist_ok=True)

# Save the model
model.save(model_path)
print(f"Model saved to {model_path}")


Model saved to /content/drive/MyDrive/Projects/ProjectMonkeyPox/Saved_Bi-lstm/bilstm_attention_model.h5
